In [2]:
# Load the dependencies
from keras.datasets import mnist
from matplotlib import pyplot

import numpy as np

import torch
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn

In [3]:
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [4]:
torch.manual_seed(0)

In [5]:
X_train[0].shape

(28, 28)

In [6]:
class Data(Dataset):
    
    #  modified from: http://cs231n.github.io/neural-networks-case-study/
    # Constructor
    def __init__(self, K=3, N=500, train=True):
        if train:
            self.y = torch.from_numpy(y_train).type(torch.LongTensor)
            self.x = torch.from_numpy(X_train).type(torch.FloatTensor)
        else:
            self.y = torch.from_numpy(y_test).type(torch.LongTensor)
            self.x = torch.from_numpy(X_test).type(torch.FloatTensor)
        
        self.len = self.y.shape[0]
    
    # Getter
    def __getitem__(self, index):    
        return self.x[index], self.y[index]
    
    # Get Length
    def __len__(self):
        return self.len
    

In [128]:
class CNN(nn.Module):
    def __init__(self, out1=1, out2=1, kernel_size=3,padding=0):
        super(CNN,self).__init__()
        
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=out1, kernel_size=kernel_size, padding=padding)
        self.relu = nn.ReLU()
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=1)
        self.linear1 = nn.Linear(in_features=625,out_features=10)
        
    def forward(self,x):
        # print(1, x.shape)
        x = self.cnn1(x)
        # print(2, x.shape)
        x = self.relu(x)
        # print(x.shape)
        x = self.maxpool1(x)
        # print(x.shape)
        x = x.view(-1, 25*25)
        #print(x.shape)
        x = self.linear1(x)
        
        y = x[0][torch.argmax(x)]
        y = y.view(-1)
        
        return y
        

In [129]:
data_set = Data()
cnn = CNN()

In [130]:
# Define the function for training the model

def train(data_set, model, criterion, train_loader, optimizer, epochs=100):
    LOSS = []
    ACC = []
    for epoch in range(epochs):
        for x, y in train_loader:
            optimizer.zero_grad()
            yhat = model(x.unsqueeze(1)).detach()
            print("y",y,"yhat", yhat)
            loss = criterion(yhat, y)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            LOSS.append(loss.item())
        ACC.append(accuracy(model, data_set))

In [131]:
learning_rate = 0.01
optimizer = torch.optim.SGD(cnn.parameters(), lr=learning_rate)
train_loader = DataLoader(dataset=data_set, batch_size=1)
criterion = nn.CrossEntropyLoss()
LOSS = train(data_set, cnn, criterion, train_loader, optimizer, epochs=1000)

y tensor([5]) yhat tensor([107.5106])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [ ]:
type(X_train)

In [ ]:
for i,x in enumerate(X_train):
    cnn(x)
    break